<img tyle="float: right;"  src="http://minneanalytics.org/wp/wp-content/uploads/2018/04/BDT18_LP-02-02.jpg" \>

In [ ]:
from IPython.core.display import display, HTML
from IPython.display import IFrame

# Jupyter notebook

In [ ]:
IFrame('https://jupyter.org/index.html', width='100%',height='1024')

In [ ]:
IFrame('https://jupyter.readthedocs.io/en/latest/architecture/how_jupyter_ipython_work.html', width='100%',height='1024')

# Using IPython for parallel computing

The IPython engine is an extension of the IPython kernel for Jupyter. The engine listens for requests over the network, runs code, and returns results. IPython parallel extends the Jupyter messaging protocol to support native Python object serialization. When multiple engines are started, parallel and distributed computing becomes possible.

In [ ]:
IFrame('http://ipyparallel.readthedocs.io/en/latest/index.html', width='100%',height='1024')

# Before connecting remember to start the cluster!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
- start a terminal
- input the command: ipcluster start


### Connect with the cluster

In [ ]:
import ipyparallel as ipp

In [ ]:
rc = ipp.Client()
rc.ids

In [ ]:
rc[:].apply_sync(lambda : "Hello, World")

In [ ]:
%px print ('Hello world')

In [ ]:
lview = rc.load_balanced_view()

In [ ]:
lview.block = True

In [ ]:
serial_result = map(lambda x:x**10, range(32))
serial_result

In [ ]:
parallel_result = lview.map(lambda x:x**10, range(32))
parallel_result

In [ ]:
serial_result==parallel_result

In [ ]:
@lview.parallel()
def f(x):
    return x**10

In [ ]:
f.map(range(32))    # this is done in parallel

# Using IPYPARALLEL to Derive flags from a string

In [ ]:
import pandas as pd
import numpy as np
from random import shuffle
# -----------------------------------
pd.set_option('display.max_columns', 512)
pd.set_option('display.max_colwidth', 1024)
pd.set_option('display.max_rows', 512)
pd.set_option('display.width', 1024)
pd.set_option('display.max_info_rows', 512)
pd.set_option('display.expand_frame_repr', True)
pd.set_option('mode.chained_assignment','warn')
pd.set_option('precision', 2)
pd.set_option('float_format', '{:6.2f}'.format)
pd.set_option('display.notebook_repr_html', True)

## Generate random codes

In [ ]:
import tqdm
from tqdm import *
import string
import random

codes = []

for r in tqdm_notebook(range(100000)):
    col = ''
    for l in range(10):
        col += random.choice(string.ascii_uppercase)
    codes.append(col) 
    
d = {'code':codes}    

In [ ]:
df = pd.DataFrame(data=d)

In [ ]:
df.shape

In [ ]:
df.head(5)

### Export code from dataframe to a list of tuples

In [ ]:
codes = df.to_records().tolist()

In [ ]:
len(codes)

### Shuffle the python list

In [ ]:
shuffle(codes)
codes

### Connect to the python Engine(s)

In [ ]:
c = ipp.Client(timeout=30.0)
dv = c[:]
dv.block = True
c.ids

### import python libraries to each Engines

In [ ]:
with dv.sync_imports():  # your 'view' variable 
    import pandas as pd
    import os

### Ping python Engine(s)

In [ ]:
%px print('Ready!' + str(os.getpid()))

### Define processor python function

In [ ]:
def processor(code):
    k = code[0]
    v = code[1]
    # ------------------------------------- #    
    # instantiate a pandas series object    #
    # ------------------------------------- #
    s0 = pandas.Series(name=k)

    # ------------------------------------- #    
    # for each letter create a flag         #
    # ------------------------------------- #
    for c in v:
        s0['code_' + c] = 1

    return s0

### Push the processor python function to each engine

In [ ]:
dv.push(dict(prc = processor))

### Partition the python list based on # Engine

In [ ]:
dv.scatter('codes',codes)

### Run python list expression in each Engine

In [ ]:
%%px --group-outputs=engine
print("Engine:{0} busy processing {1} codes".format(os.getpid(),len(codes)))
engine_result = [prc(i) for i in codes]
print("Engine:{0} ready".format(os.getpid()))

### Gather the result returned by each Engine

In [ ]:
results = dv.gather('engine_result')

### Load results to the flags dataframe

In [ ]:
df_flags = pd.DataFrame()
df_flags = df_flags.append(results)
df_flags.fillna(0, inplace=True)
df_flags = df_flags.astype(np.int64)
df_flags = df_flags[df_flags.columns.sort_values().tolist()]
df_flags.head(10)

### Join the flags dataframe with the source dataframe

In [ ]:
df_final = df.join(df_flags,how='inner')

# Data is ready!!!

In [ ]:
df_final.head(10)

In [ ]:
len(df_final)

# Questions?